<a href="https://colab.research.google.com/github/HumbertoDiego/pre-compiled-colmap-in-colab/blob/main/colmap_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Time everything

In [ ]:
!pip install ipython-autotime
%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.1 MB/s eta 0:00:00
time: 306 µs (started: 2024-12-28 00:55:18 +00:00)


## Check Cuda and environment

In [ ]:
!nvidia-smi
!nvcc --version
!gcc --version
!cat /etc/os-release | grep VERSION
!env | grep CUDA

Sat Dec 28 01:00:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Installation

In [ ]:
!sudo apt update
!sudo apt-get install \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-regex-dev \
    libboost-system-dev \
    libflann-dev \
    libboost-test-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libfreeimage-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libmetis-dev \
    libcgal-qt5-dev \
    libgtest-dev \
    libgmock-dev \
    libsqlite3-dev \
    libcurl4-openssl-dev \
    nvidia-cuda-toolkit \
    nvidia-cuda-toolkit-gcc \
    libglvnd-dev

!sudo apt install libceres-dev # directly like this have no cuda support
# Requested to use GPU for bundle adjustment, but Ceres was compiled without CUDA support.
# Falling back to CPU-based dense solvers.

Build Colmap 3.12

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

working_dir = "/content/drive/MyDrive/Colab Notebooks"
os.chdir(working_dir)
if not os.path.exists("colmap"):
  !git clone https://github.com/colmap/colmap.git
%cd colmap
if not os.path.exists("build"):
  !mkdir build
%cd build
if not os.path.exists("install_manifest.txt"):
  !cmake .. -GNinja -DCMAKE_CUDA_ARCHITECTURES=75 -DGUI_ENABLED=OFF \
            -DOPENGL_ENABLED=OFF -DTESTS_ENABLED=OFF
  !sudo chown -R $(whoami) .
  !ninja -j2
  # time: 32min 40s
!sudo ninja install

# Run built colmap-3.12.0.dev0

In [ ]:
!colmap -h

COLMAP 3.12.0.dev0 -- Structure-from-Motion and Multi-View Stereo
(Commit cb02ca13 on 2024-12-27 with CUDA)

Usage:
  colmap [command] [options]

Documentation:
  https://colmap.github.io/

Example usage:
  colmap help [ -h, --help ]
  colmap gui
  colmap gui -h [ --help ]
  colmap automatic_reconstructor -h [ --help ]
  colmap automatic_reconstructor --image_path IMAGES --workspace_path WORKSPACE
  colmap feature_extractor --image_path IMAGES --database_path DATABASE
  colmap exhaustive_matcher --database_path DATABASE
  colmap mapper --image_path IMAGES --database_path DATABASE --output_path MODEL
  ...

Available commands:
  help
  gui
  automatic_reconstructor
  bundle_adjuster
  color_extractor
  database_cleaner
  database_creator
  database_merger
  delaunay_mesher
  exhaustive_matcher
  feature_extractor
  feature_importer
  hierarchical_mapper
  image_deleter
  image_filterer
  image_rectifier
  image_registrator
  image_undistorter
  image_undistorter_standalone
  mapper
  ma

Reconstruct SPARSE model

In [ ]:
os.environ['DATASET_PATH'] = "/content/drive/MyDrive/Classroom/GS/data"

In [ ]:
# The project folder must contain a folder "images" with all the images.
!colmap feature_extractor \
   --database_path "$DATASET_PATH/train/database.db" \
   --image_path "$DATASET_PATH/train/images"
# time: 32.2 s

I1228 01:06:19.842751  6359 misc.cc:44] 
Feature extraction
I1228 01:06:19.843255  6362 sift.cc:721] Creating SIFT GPU feature extractor
I1228 01:06:21.241303  6363 feature_extraction.cc:258] Processed file [1/301]
I1228 01:06:21.241336  6363 feature_extraction.cc:261]   Name:            00001.jpg
I1228 01:06:21.241343  6363 feature_extraction.cc:270]   Dimensions:      980 x 545
I1228 01:06:21.241349  6363 feature_extraction.cc:273]   Camera:          #1 - SIMPLE_RADIAL
I1228 01:06:21.241361  6363 feature_extraction.cc:276]   Focal Length:    1176.00px
I1228 01:06:21.241376  6363 feature_extraction.cc:280]   Features:        3847
I1228 01:06:27.503962  6363 feature_extraction.cc:258] Processed file [2/301]
I1228 01:06:27.504011  6363 feature_extraction.cc:261]   Name:            00002.jpg
I1228 01:06:27.504022  6363 feature_extraction.cc:270]   Dimensions:      980 x 545
I1228 01:06:27.504031  6363 feature_extraction.cc:273]   Camera:          #2 - SIMPLE_RADIAL
I1228 01:06:27.504040 

In [ ]:
!colmap exhaustive_matcher \
   --database_path "$DATASET_PATH/train/database.db"
# time: 5min 10s

I1228 01:06:48.598628  7286 misc.cc:44] 
Feature matching
I1228 01:06:48.610080  7287 sift.cc:1426] Creating SIFT GPU feature matcher
I1228 01:06:48.713739  7286 pairing.cc:168] Generating exhaustive image pairs...
I1228 01:06:48.713773  7286 pairing.cc:201] Matching block [1/7, 1/7]
I1228 01:06:55.298846  7286 feature_matching.cc:46] in 6.585s
I1228 01:06:55.689291  7286 pairing.cc:201] Matching block [1/7, 2/7]
I1228 01:07:06.207535  7286 feature_matching.cc:46] in 10.518s
I1228 01:07:06.243193  7286 pairing.cc:201] Matching block [1/7, 3/7]
I1228 01:07:15.133001  7286 feature_matching.cc:46] in 8.890s
I1228 01:07:15.184340  7286 pairing.cc:201] Matching block [1/7, 4/7]
I1228 01:07:24.530351  7286 feature_matching.cc:46] in 9.346s
I1228 01:07:25.164755  7286 pairing.cc:201] Matching block [1/7, 5/7]
I1228 01:07:37.201781  7286 feature_matching.cc:46] in 12.037s
I1228 01:07:37.312839  7286 pairing.cc:201] Matching block [1/7, 6/7]
I1228 01:07:44.364507  7286 feature_matching.cc:46] i

In [ ]:
!mkdir $DATASET_PATH/train/sparse
!colmap mapper \
    --database_path "$DATASET_PATH/train/database.db" \
    --image_path "$DATASET_PATH/train/images" \
    --output_path "$DATASET_PATH/train/sparse"
# time: 23min 30s

I1228 01:12:40.251684  8759 incremental_pipeline.cc:237] Loading database
I1228 01:12:40.370164  8759 database_cache.cc:66] Loading cameras...
I1228 01:12:40.374567  8759 database_cache.cc:76]  301 in 0.004s
I1228 01:12:40.374593  8759 database_cache.cc:84] Loading matches...
I1228 01:12:40.704741  8759 database_cache.cc:89]  16595 in 0.330s
I1228 01:12:40.704787  8759 database_cache.cc:105] Loading images...
I1228 01:12:41.046789  8759 database_cache.cc:153]  301 in 0.342s (connected 301)
I1228 01:12:41.046896  8759 database_cache.cc:164] Loading pose priors...
I1228 01:12:41.048040  8759 database_cache.cc:175]  0 in 0.001s
I1228 01:12:41.048071  8759 database_cache.cc:184] Building correspondence graph...
I1228 01:12:41.737679  8759 database_cache.cc:210]  in 0.690s (ignored 0)
I1228 01:12:41.740619  8759 timer.cc:91] Elapsed time: 0.023 [minutes]
I1228 01:12:41.777964  8759 incremental_pipeline.cc:283] Finding good initial image pair
I1228 01:12:42.132237  8759 incremental_pipeline.

Automatically reconstruct SPARSE and DENSE model

In [ ]:
######## Automatically reconstruct SPARSE and DENSE model ################
# !colmap automatic_reconstructor --workspace_path "$DATASET_PATH/train" \
#     --image_path "$DATASET_PATH/train/images"
# time: 2h 49min 9s (started: 2024-12-27 19:14:21 +00:00)